In [ ]:
#!pip install transformers
#!pip install torch
#!pip install pandas

In [45]:
# Import required libraries
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from tqdm import tqdm

In [57]:
df=pd.read_excel('new_qa.xlsx')
df = df.rename(columns={"target_text":"label", "source_text":"text"})

In [24]:
q=['Give me the SNP-disease/trait associations of the following paragraph in JSON format (rsID, disease, allele, genotype, odds ratio, p-value, ethnicity, sex,...):']

In [46]:
q=q*df.shape[0]

,context,answer,question
0,Three of the four SNPs were significantly ass...,"[ { ""genotype/allele"": ""-278 A/G"", ""rsID"": nul...",Give me the SNP-disease/trait associations of ...
1,None of the variant alleles at any of these t...,"[ { ""genotype/allele"": ""C"", ""rsID"": ""rs3781117...",Give me the SNP-disease/trait associations of ...
2,"Of the additional SNPs genotyped, rs4899445 d...","[\n{\n""genotype/allele"": null,\n""rsID"": ""rs489...",Give me the SNP-disease/trait associations of ...
3,The variant genotypes at both SNP loci were s...,"[\n{\n""genotype/allele"": null,\n""rsID"": ""rs125...",Give me the SNP-disease/trait associations of ...
4,"In SNP rs5498 E469(A/G), similar results were...","[\n{\n""genotype/allele"": ""AA"",\n""rsID"": ""rs549...",Give me the SNP-disease/trait associations of ...
...,...,...,...
95,"However, the variant rs1052700*A was associat...","[\n{\n""genotype/allele"": ""A"",\n""rsID"": ""rs1052...",Give me the SNP-disease/trait associations of ...
96,The rs401681[C] allele has been positively as...,"[\n{\n""genotype/allele"": ""C"",\n""rsID"": ""rs4016...",Give me the SNP-disease/trait associations of ...
97,In the MSA-C subgroup the association strengt...,"[\n{\n""genotype/allele"": null,\n""rsID"": ""rs382...",Give me the SNP-disease/trait associations of ...
98,We repeated the analysis in the MSA-C subgrou...,"[\n{\n""genotype/allele"": null,\n""rsID"": ""rs382...",Give me the SNP-disease/trait associations of ...


In [38]:
df

,context,answer,question
0,Three of the four SNPs were significantly ass...,"[ { ""genotype/allele"": ""-278 A/G"", ""rsID"": nul...",Give me the SNP-disease/trait associations of ...
1,None of the variant alleles at any of these t...,"[ { ""genotype/allele"": ""C"", ""rsID"": ""rs3781117...",Give me the SNP-disease/trait associations of ...
2,"Of the additional SNPs genotyped, rs4899445 d...","[\n{\n""genotype/allele"": null,\n""rsID"": ""rs489...",Give me the SNP-disease/trait associations of ...
3,The variant genotypes at both SNP loci were s...,"[\n{\n""genotype/allele"": null,\n""rsID"": ""rs125...",Give me the SNP-disease/trait associations of ...
4,"In SNP rs5498 E469(A/G), similar results were...","[\n{\n""genotype/allele"": ""AA"",\n""rsID"": ""rs549...",Give me the SNP-disease/trait associations of ...
...,...,...,...
95,"However, the variant rs1052700*A was associat...","[\n{\n""genotype/allele"": ""A"",\n""rsID"": ""rs1052...",Give me the SNP-disease/trait associations of ...
96,The rs401681[C] allele has been positively as...,"[\n{\n""genotype/allele"": ""C"",\n""rsID"": ""rs4016...",Give me the SNP-disease/trait associations of ...
97,In the MSA-C subgroup the association strengt...,"[\n{\n""genotype/allele"": null,\n""rsID"": ""rs382...",Give me the SNP-disease/trait associations of ...
98,We repeated the analysis in the MSA-C subgrou...,"[\n{\n""genotype/allele"": null,\n""rsID"": ""rs382...",Give me the SNP-disease/trait associations of ...


In [39]:
df['question']=q

In [40]:
class QADataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        context = self.df.iloc[idx]['context']
        question = self.df.iloc[idx]['question']
        answer = self.df.iloc[idx]['answer']
        
        encoding = self.tokenizer.encode_plus(question, context, return_tensors='pt')
        input_ids = encoding['input_ids'].squeeze()
        token_type_ids = encoding['token_type_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()
        
        start_pos, end_pos = self.find_answer_indices(context, answer)
        
        return {'input_ids': input_ids, 
                'token_type_ids': token_type_ids, 
                'attention_mask': attention_mask,
                'start_positions': start_pos, 
                'end_positions': end_pos}
    
    def find_answer_indices(self, context, answer):
        start_pos = context.find(answer)
        end_pos = start_pos + len(answer) - 1
        return start_pos, end_pos
 
 
 


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

dataset = QADataset(df, tokenizer)
 

In [41]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])


In [42]:
from_pretrained('bert-base-uncased')

In [33]:
df.columns

Index(['context', 'answer', 'question'], dtype='object')

In [80]:
df['label'].iloc[0].split(",")[0]

'[ { "genotype/allele": "-278 A/G"'

In [82]:
import pandas as pd
from sentence_transformers import SentenceTransformer, InputExample
from sentence_transformers.evaluation import SentenceEvaluator

# Load your dataset
#df = pd.read_csv('your_dataset.csv')

# Create a list of InputExamples from your dataset
examples = []
for index, row in df.iterrows():
    examples.append(InputExample(texts=[row['text']], label=row['label'].split(",")))

# Create the SentenceTransformer model
model = SentenceTransformer( 'emilyalsentzer/Bio_ClinicalBERT')

# Train the model on your dataset
model.fit(examples)

# Evaluate the model
evaluator = SentenceEvaluator(examples)
evaluator.evaluate(model)

No sentence-transformers model found with name C:\Users\AlexandruDaia/.cache\torch\sentence_transformers\emilyalsentzer_Bio_ClinicalBERT. Creating a new one with MEAN pooling.
loading configuration file C:\Users\AlexandruDaia/.cache\torch\sentence_transformers\emilyalsentzer_Bio_ClinicalBERT\config.json
Model config BertConfig {
  "_name_or_path": "C:\\Users\\AlexandruDaia/.cache\\torch\\sentence_transformers\\emilyalsentzer_Bio_ClinicalBERT",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file C:\Users\AlexandruD

TypeError: cannot unpack non-iterable InputExample object

In [83]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

model = AutoModelForQuestionAnswering.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

model.train()
model.fit(df, df)

https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT/resolve/main/config.json not found in cache or force_download set to True, downloading to C:\Users\AlexandruDaia\.cache\huggingface\transformers\tmpenbvp_61


Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

storing https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT/resolve/main/config.json in cache at C:\Users\AlexandruDaia/.cache\huggingface\transformers\dc6d60ebe42d83e1479ce0d473758bb3586763ff6c4c814bda5321acf856bd64.b74d0770929e519c6d193d16b6874051ae549f5c8c228903a48e59d36260466b
creating metadata file for C:\Users\AlexandruDaia/.cache\huggingface\transformers\dc6d60ebe42d83e1479ce0d473758bb3586763ff6c4c814bda5321acf856bd64.b74d0770929e519c6d193d16b6874051ae549f5c8c228903a48e59d36260466b
loading configuration file https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT/resolve/main/config.json from cache at C:\Users\AlexandruDaia/.cache\huggingface\transformers\dc6d60ebe42d83e1479ce0d473758bb3586763ff6c4c814bda5321acf856bd64.b74d0770929e519c6d193d16b6874051ae549f5c8c228903a48e59d36260466b
Model config BertConfig {
  "_name_or_path": "emilyalsentzer/Bio_ClinicalBERT",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob":

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

storing https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT/resolve/main/pytorch_model.bin in cache at C:\Users\AlexandruDaia/.cache\huggingface\transformers\794538e7c825dc7be96d9fc3c73b79a9736da5f699fc50d31513dbca0740b349.f0d8b668347b3048f5b88e273fde3c3412366726bc99aa5935b7990944092fb1
creating metadata file for C:\Users\AlexandruDaia/.cache\huggingface\transformers\794538e7c825dc7be96d9fc3c73b79a9736da5f699fc50d31513dbca0740b349.f0d8b668347b3048f5b88e273fde3c3412366726bc99aa5935b7990944092fb1
loading weights file https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT/resolve/main/pytorch_model.bin from cache at C:\Users\AlexandruDaia/.cache\huggingface\transformers\794538e7c825dc7be96d9fc3c73b79a9736da5f699fc50d31513dbca0740b349.f0d8b668347b3048f5b88e273fde3c3412366726bc99aa5935b7990944092fb1
Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.weight', 'cls.predictions.t

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

storing https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT/resolve/main/vocab.txt in cache at C:\Users\AlexandruDaia/.cache\huggingface\transformers\d544709b8432c5d7b9fef7d9361a9cc048632d93776ae61e401a6eff0fb8f037.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791
creating metadata file for C:\Users\AlexandruDaia/.cache\huggingface\transformers\d544709b8432c5d7b9fef7d9361a9cc048632d93776ae61e401a6eff0fb8f037.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791
loading file https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT/resolve/main/vocab.txt from cache at C:\Users\AlexandruDaia/.cache\huggingface\transformers\d544709b8432c5d7b9fef7d9361a9cc048632d93776ae61e401a6eff0fb8f037.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791
loading file https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT/resolve/main/tokenizer.json from cache at None
loading file https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT/resolve/main/added_tokens.json f

AttributeError: 'BertForQuestionAnswering' object has no attribute 'fit'

In [85]:
model.train(df)

ValueError: training mode is expected to be boolean

In [86]:
df

,text,label
0,Three of the four SNPs were significantly ass...,"[ { ""genotype/allele"": ""-278 A/G"", ""rsID"": nul..."
1,None of the variant alleles at any of these t...,"[ { ""genotype/allele"": ""C"", ""rsID"": ""rs3781117..."
2,"Of the additional SNPs genotyped, rs4899445 d...","[\n{\n""genotype/allele"": null,\n""rsID"": ""rs489..."
3,The variant genotypes at both SNP loci were s...,"[\n{\n""genotype/allele"": null,\n""rsID"": ""rs125..."
4,"In SNP rs5498 E469(A/G), similar results were...","[\n{\n""genotype/allele"": ""AA"",\n""rsID"": ""rs549..."
...,...,...
95,"However, the variant rs1052700*A was associat...","[\n{\n""genotype/allele"": ""A"",\n""rsID"": ""rs1052..."
96,The rs401681[C] allele has been positively as...,"[\n{\n""genotype/allele"": ""C"",\n""rsID"": ""rs4016..."
97,In the MSA-C subgroup the association strengt...,"[\n{\n""genotype/allele"": null,\n""rsID"": ""rs382..."
98,We repeated the analysis in the MSA-C subgrou...,"[\n{\n""genotype/allele"": null,\n""rsID"": ""rs382..."


In [96]:
 
import tensorflow as tf
import pandas as pd
import numpy as np
from transformers import TFBertForSequenceClassification, BertTokenizer

# Load the BioBERT pre-trained model and tokenizer
model = TFBertForSequenceClassification.from_pretrained('dmis-lab/biobert-v1.1', num_labels=2, from_pt=True)
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-v1.1')

# Load the training data
train_data = df

# Tokenize the input text and convert it into numerical tokens
input_ids = []
attention_masks = []
for text in train_data['text']:
    encoded_dict = tokenizer.encode_plus(
                        text,                      # Text to encode
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,          # Max length of the text that can go to BERT
                        pad_to_max_length = True,  # Pad the rest of the sequence to max length
                        return_attention_mask = True, # Return attention mask
                        return_tensors = 'tf',     # Return TensorFlow tensors
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the labels to numerical format
labels = np.array(train_data['label'])

# Create a TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((input_ids, attention_masks, labels)).batch(32)

# Define the optimizer, loss function, and metrics
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

# Define the training loop
@tf.function
def train_step(input_ids, attention_masks, labels):
    with tf.GradientTape() as tape:
        logits = model([input_ids, attention_masks])[0]
        loss_value = loss(labels, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    metric.update_state(labels, logits)
    return loss_value

# Train the model
for epoch in range(5):
    for step, batch in enumerate(dataset):
        loss_value = train_step(batch[0], batch[1], batch[2])
        if step % 10 == 0:
            print(f'Epoch {epoch}, Step {step}, Loss {loss_value}, Accuracy {metric.result()}')
            metric.reset_states()

# Save the fine-tuned model
model.save_pretrained('bio_bert_summarization')

loading configuration file https://huggingface.co/dmis-lab/biobert-v1.1/resolve/main/config.json from cache at C:\Users\AlexandruDaia/.cache\huggingface\transformers\f048b8136bae2b3abe91e9e82949295fb205887c84db3be2775e1cdb0ecfeeb9.d7812d36d3371e4d43299a0c4a938622c5251db0efa17a5d4d9b57037fcec823
Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

https://huggingface.co/dmis-lab/biobert-v1.1/resolve/main/pytorch_model.bin not found in 

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

storing https://huggingface.co/dmis-lab/biobert-v1.1/resolve/main/pytorch_model.bin in cache at C:\Users\AlexandruDaia/.cache\huggingface\transformers\65231a5792b14eb81b9a6bdccccfffda18575eb3bafbb730c9fa4235e56c3c17.74cc2087932cb523a583bd5e65732ee1aaade59dfc0b62f88101de7567d92e42
creating metadata file for C:\Users\AlexandruDaia/.cache\huggingface\transformers\65231a5792b14eb81b9a6bdccccfffda18575eb3bafbb730c9fa4235e56c3c17.74cc2087932cb523a583bd5e65732ee1aaade59dfc0b62f88101de7567d92e42
loading weights file https://huggingface.co/dmis-lab/biobert-v1.1/resolve/main/pytorch_model.bin from cache at C:\Users\AlexandruDaia/.cache\huggingface\transformers\65231a5792b14eb81b9a6bdccccfffda18575eb3bafbb730c9fa4235e56c3c17.74cc2087932cb523a583bd5e65732ee1aaade59dfc0b62f88101de7567d92e42
Loading PyTorch weights from C:\Users\AlexandruDaia\.cache\huggingface\transformers\65231a5792b14eb81b9a6bdccccfffda18575eb3bafbb730c9fa4235e56c3c17.74cc2087932cb523a583bd5e65732ee1aaade59dfc0b62f88101de7567d92e

ValueError: in user code:

    File "C:\Users\AlexandruDaia\AppData\Local\Temp\ipykernel_10336\2161090654.py", line 43, in train_step  *
        logits = model([input_ids, attention_masks])[0]
    File "C:\Users\AlexandruDaia\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\ALEXAN~1\AppData\Local\Temp\__autograph_generated_file_3dmposs.py", line 17, in tf__call
        outputs = ag__.converted_call(ag__.ld(self).bert, (), dict(input_ids=ag__.ld(inputs)['input_ids'], attention_mask=ag__.ld(inputs)['attention_mask'], token_type_ids=ag__.ld(inputs)['token_type_ids'], position_ids=ag__.ld(inputs)['position_ids'], head_mask=ag__.ld(inputs)['head_mask'], inputs_embeds=ag__.ld(inputs)['inputs_embeds'], output_attentions=ag__.ld(inputs)['output_attentions'], output_hidden_states=ag__.ld(inputs)['output_hidden_states'], return_dict=ag__.ld(inputs)['ret

In [91]:
df

,text,label
0,Three of the four SNPs were significantly ass...,"[ { ""genotype/allele"": ""-278 A/G"", ""rsID"": nul..."
1,None of the variant alleles at any of these t...,"[ { ""genotype/allele"": ""C"", ""rsID"": ""rs3781117..."
2,"Of the additional SNPs genotyped, rs4899445 d...","[\n{\n""genotype/allele"": null,\n""rsID"": ""rs489..."
3,The variant genotypes at both SNP loci were s...,"[\n{\n""genotype/allele"": null,\n""rsID"": ""rs125..."
4,"In SNP rs5498 E469(A/G), similar results were...","[\n{\n""genotype/allele"": ""AA"",\n""rsID"": ""rs549..."
...,...,...
95,"However, the variant rs1052700*A was associat...","[\n{\n""genotype/allele"": ""A"",\n""rsID"": ""rs1052..."
96,The rs401681[C] allele has been positively as...,"[\n{\n""genotype/allele"": ""C"",\n""rsID"": ""rs4016..."
97,In the MSA-C subgroup the association strengt...,"[\n{\n""genotype/allele"": null,\n""rsID"": ""rs382..."
98,We repeated the analysis in the MSA-C subgrou...,"[\n{\n""genotype/allele"": null,\n""rsID"": ""rs382..."
